# Lecture 18

## Cross Validation and Regularization

In this section we will work through the train test-split and the process of cross validation.  

## Imports

As with other notebooks we will use the same set of standard imports.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
#import cufflinks as cf
#cf.set_config_file(offline=True, sharing=False, theme='ggplot');

In [3]:
from sklearn.linear_model import LinearRegression

## The Data

For this notebook, we will use the seaborn `mpg` dataset which describes the fuel mileage (measured in miles per gallon or mpg) of various cars along with characteristics of those cars.  Our goal will be to build a model that can predict the fuel mileage of a car based on the characteristics of that car.

In [4]:
from seaborn import load_dataset
data = load_dataset("mpg")
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,usa,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,europe,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,usa,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,usa,ford ranger


This data has some rows with missing data. We will ignore those rows until later for the sake of this lecture. We can use the Pandas `DataFrame.isna` function to find rows with missing values and drop them, although of course, this is not always the best idea!

In [10]:
data = data[~data.isna().any(axis=1)].copy()

### Using SKLearn

We can use the `train_test_split` function from `sklearn.model_selection` to do this easily.

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
tr, te = train_test_split(data, test_size = 0.1,random_state=83)

In [12]:
len(tr), len(te)

(352, 40)

In [13]:
tr.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
6,14.0,8,454.0,220.0,4354,9.0,70,usa,chevrolet impala
352,29.9,4,98.0,65.0,2380,20.7,81,usa,ford escort 2h
47,19.0,6,250.0,100.0,3282,15.0,71,usa,pontiac firebird
39,14.0,8,400.0,175.0,4464,11.5,71,usa,pontiac catalina brougham
304,37.3,4,91.0,69.0,2130,14.7,79,europe,fiat strada custom


In [14]:
te.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
87,13.0,8,350.0,145.0,3988,13.0,73,usa,chevrolet malibu
279,29.5,4,98.0,68.0,2135,16.6,78,japan,honda accord lx
319,31.3,4,120.0,75.0,2542,17.5,80,japan,mazda 626
173,24.0,4,119.0,97.0,2545,17.0,75,japan,datsun 710
148,26.0,4,116.0,75.0,2246,14.0,74,europe,fiat 124 tc


## Building A Basic Model

Let's go through the process of building a model. Let's start by looking at the raw quantitative features available. We will first use just our own feature function (as we did in previous lectures). This function will just extract the quantitative features we can use for our model.

In [15]:
def basic_design_matrix(df):
    X = df[["cylinders", "displacement", 
          "horsepower", "weight", "acceleration", "model_year"]]
    return X

basic_design_matrix(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year
6,8,454.0,220.0,4354,9.0,70
352,4,98.0,65.0,2380,20.7,81
47,6,250.0,100.0,3282,15.0,71
39,8,400.0,175.0,4464,11.5,71
304,4,91.0,69.0,2130,14.7,79
...,...,...,...,...,...,...
394,4,97.0,52.0,2130,24.6,82
258,6,231.0,105.0,3380,15.8,78
297,5,183.0,77.0,3530,20.1,79
23,4,121.0,113.0,2234,12.5,70


Then we fit a `scikit-learn` `LinearRegression` model to our training data.

In [16]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [17]:
model.fit(basic_design_matrix(tr), tr['mpg'])

LinearRegression()

To evaluate the error we will use the **Root Mean Squared Error (RMSE)** which is like the mean squared error but in the correct units (mpg) instead of (mpg^2). 

In [18]:
def rmse(y, yhat):
    return np.sqrt(np.mean((y - yhat)**2))

The training error is:

In [19]:
Y_hat = model.predict(basic_design_matrix(tr))
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 3.3745826999424584


Oh no! We just used the test data to evaluate our model! We shouldn't have done that.  

(Don't worry, this is only for demonstration purposes. But seriously, don't try this at home.)  

**Notice:** The test error is slightly higher than the training error. This is typically (but not always) the case. Sometimes we get lucky and the test data is "easier to predict" or happens to closely follow the training data.

## Cross-Validation

#### Keeping track of all the models.

In this notebook we will want to keep track of all our models. 

We store our model settings in a dictionary. The key is some identifying name, and the value is a 2-element tuple, with the first element being the transformation function (e.g. `basic_design_matrix`), and the second element being an empty model object (e.g. `LinearRegression()`).

In [20]:
models = {"quant": (basic_design_matrix, LinearRegression())}

### More Feature Transformations

As in previous lecture, we might want to look at the displacement per cylinder as well.

In [21]:
def dispcyl_design_matrix(df):
    X = basic_design_matrix(df)
    X['displacement/cylinder'] = X['displacement'] / X['cylinders']
    return X

dispcyl_design_matrix(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year,displacement/cylinder
6,8,454.0,220.0,4354,9.0,70,56.750000
352,4,98.0,65.0,2380,20.7,81,24.500000
47,6,250.0,100.0,3282,15.0,71,41.666667
39,8,400.0,175.0,4464,11.5,71,50.000000
304,4,91.0,69.0,2130,14.7,79,22.750000
...,...,...,...,...,...,...,...
394,4,97.0,52.0,2130,24.6,82,24.250000
258,6,231.0,105.0,3380,15.8,78,38.500000
297,5,183.0,77.0,3530,20.1,79,36.600000
23,4,121.0,113.0,2234,12.5,70,30.250000


We can build a linear model using the same quantitative features as before, but with this new "displacement per cylinder" feature.

In [22]:
model = LinearRegression()
model.fit(dispcyl_design_matrix(tr), tr['mpg'])

models['quant+dc'] = (dispcyl_design_matrix, LinearRegression())

In [23]:
Y_hat = model.predict(dispcyl_design_matrix(tr))
Y = tr['mpg']
print("Training Error (RMSE):", rmse(Y, Y_hat))

Training Error (RMSE): 3.033309344625912


Our training error is definitely lower than with the previous model, but what we really care about is the model's performance on new data. But we shouldn't actually ever look at the test data. Instead, to compare these models, we can use cross-validation to "mimic" the train-test split.

In the following function we use the sklearn `KFold` cross validation class. 

Here we define a five fold cross validation with 

```python 
five_fold = KFold(n_splits=5)
```

Then we loop over the 5 splits and get the indicies (`tr_ind`) in the training data to use for training and the indices (`va_ind`) in the training data to use for validation:

```python
for tr_ind, te_ind in five_fold.split(tr):
```

In [24]:
from sklearn.model_selection import KFold
from sklearn.base import clone

def cross_validate_rmse(phi_function, model):
    model = clone(model)
    five_fold = KFold(n_splits = 5, random_state = 100, shuffle = True)
    rmse_values = []
    for tr_ind, va_ind in five_fold.split(tr):
        
        X_train = phi_function(tr.iloc[tr_ind, :])
        y_train = tr['mpg'].iloc[tr_ind]
        X_val = phi_function(tr.iloc[va_ind, :])
        y_val = tr['mpg'].iloc[va_ind]
        
        model.fit(X_train, y_train)
        
        rmse_values.append(rmse(y_val, model.predict(X_val)))
        
    return np.mean(rmse_values)


Valiating the model

In [25]:
cross_validate_rmse(dispcyl_design_matrix, LinearRegression())

3.111315976504573

The following helper function generates a plot comparing all the models in the `transformations` dictionary.

In [26]:
def compare_models(models):
    
    # Compute the training error for each model
    training_rmse = []
    for transformation, model in models.values():
        model = clone(model)
        model.fit(transformation(tr), tr['mpg'])
        training_rmse.append(rmse(tr['mpg'], model.predict(transformation(tr))))
    
    # Compute the cross validation error for each model
    validation_rmse = [cross_validate_rmse(transformation, model) for transformation, model in models.values()]
    
    names = list(models.keys())
    fig = go.Figure([
        go.Bar(x = names, y = training_rmse, name="Training RMSE"),
        go.Bar(x = names, y = validation_rmse, name="CV RMSE")])
    return fig

In [30]:
fig = compare_models(models)
fig.update_yaxes(range = [0, 4], title = "RMSE")

So not only did the new displacement / cylinders feature improve our training error, it also improved our cross-validation error. This indicates that this feature helps our model generalize, or in other words, that it has "predictive power."

Now let's try adding some categorical data, such as the `origin` column. As this is categorical data, we will have to one-hot encode this variable.

In [38]:
data['origin'].value_counts()

usa       245
japan      79
europe     68
Name: origin, dtype: int64

Fortunately, it looks like we have only three possible values for `origin`. We will use `scikit-learn`'s one-hot encoder to do the transformation. Check out Lecture 16 for a refresher on how this works.

In [39]:
from sklearn.preprocessing import OneHotEncoder
oh_enc = OneHotEncoder()
oh_enc.fit(data[['origin']])

def origin_design_matrix(df):
    X = dispcyl_design_matrix(df)
    ohe_cols = pd.DataFrame(oh_enc.transform(df[['origin']]).todense(), 
                           columns = oh_enc.get_feature_names_out(),
                           index = df.index)
    return X.join(ohe_cols)

models['quant+dc+o'] = (origin_design_matrix, LinearRegression())

origin_design_matrix(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year,displacement/cylinder,origin_europe,origin_japan,origin_usa
6,8,454.0,220.0,4354,9.0,70,56.750000,0.0,0.0,1.0
352,4,98.0,65.0,2380,20.7,81,24.500000,0.0,0.0,1.0
47,6,250.0,100.0,3282,15.0,71,41.666667,0.0,0.0,1.0
39,8,400.0,175.0,4464,11.5,71,50.000000,0.0,0.0,1.0
304,4,91.0,69.0,2130,14.7,79,22.750000,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
394,4,97.0,52.0,2130,24.6,82,24.250000,1.0,0.0,0.0
258,6,231.0,105.0,3380,15.8,78,38.500000,0.0,0.0,1.0
297,5,183.0,77.0,3530,20.1,79,36.600000,1.0,0.0,0.0
23,4,121.0,113.0,2234,12.5,70,30.250000,1.0,0.0,0.0


In [40]:
fig = compare_models(models)
fig.update_yaxes(range = [0, 4], title = "RMSE")

It looks like adding these new features about origin didn't really affect our model.

Let's try if we can gain any information from the `name` column. This column contains the make and model of each car. The models are fairly unique, so let's try to extract information about the brand (e.g. `ford`). The following cell shows the top 20 words that appear in this column.

In [37]:
tr['name'].str.split().explode().value_counts().head(20)

ford          44
chevrolet     37
plymouth      28
(sw)          27
amc           26
dodge         26
toyota        22
datsun        17
custom        17
buick         16
volkswagen    14
pontiac       14
oldsmobile    10
mercury       10
rabbit        10
honda         10
brougham      10
corolla        9
mazda          9
colt           7
Name: name, dtype: int64

It looks like there is at least one model here (`corolla`), but it does show the most common brands. We will add one column for each of these strings, with a `1` for a specific car indicating that the name of the car contains the string.

Note: In practice, you would use `scikit-learn` or some other package, but we will do this manually just to be explicit about what we're doing.

In [41]:
brands = tr['name'].str.split().explode().value_counts().head(20).index

def brands_design_matrix(df):
    X = origin_design_matrix(df)
    for brand in brands:
        X[brand] = df['name'].str.contains(brand, regex = False).astype(float)
    return X

models['quant+dc+o+b'] = (brands_design_matrix, LinearRegression())

brands_design_matrix(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year,displacement/cylinder,origin_europe,origin_japan,origin_usa,...,volkswagen,pontiac,oldsmobile,mercury,rabbit,honda,brougham,corolla,mazda,colt
6,8,454.0,220.0,4354,9.0,70,56.750000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,4,98.0,65.0,2380,20.7,81,24.500000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
47,6,250.0,100.0,3282,15.0,71,41.666667,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,8,400.0,175.0,4464,11.5,71,50.000000,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
304,4,91.0,69.0,2130,14.7,79,22.750000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,4,97.0,52.0,2130,24.6,82,24.250000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
258,6,231.0,105.0,3380,15.8,78,38.500000,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,5,183.0,77.0,3530,20.1,79,36.600000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,4,121.0,113.0,2234,12.5,70,30.250000,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
fig = compare_models(models)
fig.update_yaxes(range = [0, 4], title = "RMSE")

Interesting. Adding the brand information to our design matrix decreased our training error, but it increased our cross-validation error. Looks like we overfit!

## Regularization

In this section we explore the use of regularization techniques to address overfitting.

### Ridge Regression

Ridge regression combines the ridge (L2, Squared) regularization function with the least squares loss.  

$$
\hat{\theta}_\alpha = \arg \min_\theta \left[ \left(\frac{1}{n} \sum_{i=1}^n \left(Y_i -  f_\theta(X_i)\right)^2 \right) + \alpha \sum_{k=1}^d \theta_k^2 \right]
$$
### The Regularization Hyperparameter

The $\alpha$ parameter is our regularization **hyperparameter**. It is a hyperparameter because it is not a model parameter but a choice of how we want to balance fitting the data and "over-fitting". The goal is to find a value of this hyper"parameter to maximize our accuracy on the **test data**. However, **we can't use the test data to make modeling decisions** so we turn to cross validation. The standard way to find the best $\alpha$ is to try a bunch of values and take the one with the lowest cross validation error. 


### Ridge Regression in SK Learn

Both Ridge Regression and Lasso are built-in functions in SKLearn.  Let's start by importing the `Ridge` Regression class which behaves identically to the `LinearRegression` class we used earlier:

In [43]:
from sklearn.linear_model import Ridge

Take a look at the documentation.  Notice the regularized loss function. 

In [44]:
Ridge?

Instead of just looking at the top 20 brands, let's bag-of-words encode the entire name column.

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(tr['name'])

CountVectorizer()

In [47]:
def name_design_matrix(df):
    X = origin_design_matrix(df)
    X_encoding = pd.DataFrame(
        vectorizer.transform(df['name']).todense(), 
        columns=vectorizer.get_feature_names_out(),
        index = df.index)
    return X.join(X_encoding)

name_design_matrix(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year,displacement/cylinder,origin_europe,origin_japan,origin_usa,...,volare,volkswagen,volvo,vw,wagon,woody,xe,yorker,zephyr,zx
6,8,454.0,220.0,4354,9.0,70,56.750000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
352,4,98.0,65.0,2380,20.7,81,24.500000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
47,6,250.0,100.0,3282,15.0,71,41.666667,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
39,8,400.0,175.0,4464,11.5,71,50.000000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
304,4,91.0,69.0,2130,14.7,79,22.750000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,4,97.0,52.0,2130,24.6,82,24.250000,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
258,6,231.0,105.0,3380,15.8,78,38.500000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
297,5,183.0,77.0,3530,20.1,79,36.600000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
23,4,121.0,113.0,2234,12.5,70,30.250000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
cross_validate_rmse(name_design_matrix, LinearRegression())

434576644.31252116

Woah, our RMSE is really, really large! 

To get around this, let's try regularization. As we're introducing regularization, let's also standardize our quantitative features:

In [49]:
from sklearn.preprocessing import StandardScaler
quantitative_features = ["cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year"]
scaler = StandardScaler()
scaler.fit(basic_design_matrix(tr[quantitative_features]))

StandardScaler()

In [52]:
StandardScaler.fit?

In [50]:
def name_design_matrix_std(df):
    X = name_design_matrix(df)
    X[quantitative_features] = scaler.transform(X[quantitative_features])
    return X

name_design_matrix_std(tr)

,cylinders,displacement,horsepower,weight,acceleration,model_year,displacement/cylinder,origin_europe,origin_japan,origin_usa,...,volare,volkswagen,volvo,vw,wagon,woody,xe,yorker,zephyr,zx
6,1.452785,2.426007,2.916388,1.580639,-2.320642,-1.611836,56.750000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
352,-0.874316,-0.933437,-1.034011,-0.720703,1.862364,1.351615,24.500000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
47,0.289235,0.500933,-0.141986,0.330873,-0.175511,-1.342431,41.666667,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
39,1.452785,1.916429,1.769498,1.708880,-1.426838,-1.342431,50.000000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
304,-0.874316,-0.999493,-0.932065,-1.012159,-0.282767,0.812806,22.750000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,-0.874316,-0.942873,-1.365335,-1.012159,3.256700,1.621020,24.250000,1.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
258,0.289235,0.321637,-0.014553,0.445124,0.110507,0.543401,38.500000,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
297,-0.292540,-0.131322,-0.728174,0.619998,1.647851,0.812806,36.600000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
23,-0.874316,-0.716394,0.189338,-0.890913,-1.069316,-1.611836,30.250000,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
from sklearn.linear_model import RidgeCV

In [54]:
RidgeCV?

In [55]:
# 5-fold CV
models['quant+dc+o+n-RidgeCV-5fold'] = (name_design_matrix_std, RidgeCV(alphas = np.arange(0.1,1,10),cv = 5))

In [56]:
fig = compare_models(models)
fig.update_yaxes(range = [0, 4], title = "RMSE")

### Lasso Regression

Lasso regression combines the absolute (L1) regularization function with the least squares loss.  

$$
\hat{\theta}_\alpha = \arg \min_\theta \left(\frac{1}{n} \sum_{i=1}^n \left(Y_i -  f_\theta(X_i)\right)^2 \right) + \alpha \sum_{k=1}^d |\theta_k|
$$

Lasso is actually an acronym (and a cool name) which stands for Least Absolute Shrinkage and Selection Operator.  It is an absolute operator because it is the absolute value.  It is a shrinkage operator because it favors smaller parameter values.  It is a selection operator because it has the peculiar property of pushing parameter values all the way to zero thereby selecting the remaining features.  It is this last property that makes Lasso regression so useful. By using Lasso regression and setting sufficiently large value of $\alpha$ you can eliminate features that are not informative. 

Unfortunately, there is no closed form solution for Lasso regression and so iterative optimization algorithms like gradient descent are typically used. 

In [57]:
from sklearn.linear_model import Lasso, LassoCV

In [43]:
LassoCV?

In [58]:
models['quant+dc+o+n-LassoCV'] = (name_design_matrix_std,  LassoCV(alphas = np.arange(0.1,1,0.1), cv = 5))

In [59]:
fig = compare_models(models)
fig.update_yaxes(range = [0, 4], title = "RMSE")

Let's compare the distribution of the parameters for both the `RidgeCV` and `LassoCV` models.

In [60]:
model = RidgeCV()
model.fit(name_design_matrix_std(tr), tr['mpg'])
ridge_coef = model.coef_

model = LassoCV()
model.fit(name_design_matrix_std(tr), tr['mpg'])
lasso_coef = model.coef_

In [61]:
ff.create_distplot([ridge_coef, lasso_coef], ["Ridge", "Lasso"], bin_size = 0.1)

In [62]:
name_design_matrix_std(tr).columns[lasso_coef != 0]

Index(['cylinders', 'displacement', 'horsepower', 'weight', 'model_year',
       'displacement/cylinder', 'origin_japan', 'origin_usa', 'amc', 'datsun',
       'diesel', 'ford', 'plymouth', 'pontiac', 'vw'],
      dtype='object')

In [63]:
name_design_matrix_std(tr).columns[ridge_coef != 0]

Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'model_year', 'displacement/cylinder', 'origin_europe', 'origin_japan',
       'origin_usa',
       ...
       'volare', 'volkswagen', 'volvo', 'vw', 'wagon', 'woody', 'xe', 'yorker',
       'zephyr', 'zx'],
      dtype='object', length=294)